# SETUP

## imports

In [4]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from collections import deque
from pprint import pprint
import operator
from time import sleep
import re
from functools import reduce
from dataclasses import dataclass
from copy import deepcopy
import matplotlib.pyplot as plt

## constants

In [5]:
lowercase = string.ascii_lowercase
uppercase = string.ascii_uppercase

## helpers

In [6]:
def get_level_input(lvl_num):
    with open(f"advent_inputs/{lvl_num}.txt") as f:
        level_input=f.read()
        return level_input
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# 1

## setup

In [44]:
module_weights = get_level_input("01").splitlines()
module_weights = list(map(int, module_weights))

## part one

In [45]:
%%time
total_weight = sum([x//3 - 2 for x in module_weights])
print_result(total_weight)

'RESULT: 3432671'

'TIME............................................................'
CPU times: user 727 µs, sys: 534 µs, total: 1.26 ms
Wall time: 803 µs


## part two

In [67]:
%%time

def weight_of_weight_gen(m_weight):
    while m_weight>0:
        yield m_weight
        m_weight = m_weight//3 - 2
    
def get_req_fuel(m_weight): 
    return sum([i for i in weight_of_weight_gen(m_weight)][1:])
    
total_weight = sum([get_req_fuel(x) for x in module_weights])
print_result(total_weight)

'RESULT: 5146132'

'TIME............................................................'
CPU times: user 1.14 ms, sys: 284 µs, total: 1.42 ms
Wall time: 1.19 ms


# 2

## setup

In [172]:
OPCODE = 0
INPUT_1 = 1
INPUT_2 = 2
OUTPUT = 3
WINDOW = 4

ADD = 1
MULT = 2
END = 99

comp_string = get_level_input("02")
comp_string = list(map(int, comp_string.split(',')))
# comp_string = [1,9,10,3,2,3,11,0,99,30,40,50]


## part one

In [181]:
%%time

def run_computer(comp_string):
    comp_string[1] = 12
    comp_string[2] = 2
    comp_len = len(comp_string)
    for i in range(0, len(comp_string), 4):
        win = comp_string[i:i+WINDOW]
        if(win[OPCODE] == END): return comp_string
        assert(all(x < comp_len for x in win))
        comp_string[win[OUTPUT]] = {
            ADD: comp_string[win[INPUT_1]]+comp_string[win[INPUT_2]],
            MULT: comp_string[win[INPUT_1]]*comp_string[win[INPUT_2]]
        }[win[OPCODE]]
    
print_result(run_computer(list(comp_string))[0])

'RESULT: 5290681'

'TIME............................................................'
CPU times: user 272 µs, sys: 70 µs, total: 342 µs
Wall time: 304 µs


## part two 

In [188]:
%%time

TARGET = 19690720

def run_computer(comp_string, noun, verb):
    comp_string[1] = noun 
    comp_string[2] = verb 
    comp_len = len(comp_string)
    for i in range(0, len(comp_string), 4):
        win = comp_string[i:i+WINDOW]
        if(win[OPCODE] == END): return comp_string[0]
        if(not all(x < comp_len for x in win)): return -1
        comp_string[win[OUTPUT]] = {
            ADD: comp_string[win[INPUT_1]]+comp_string[win[INPUT_2]],
            MULT: comp_string[win[INPUT_1]]*comp_string[win[INPUT_2]]
        }[win[OPCODE]]
    
def find_noun_verb(comp_string):
    for i in range(100):
        for j in range(100):
            res = run_computer(list(comp_string), i, j) 
            if res == -1: continue 
            if(res == TARGET): return i,j

noun, verb = find_noun_verb(list(comp_string))

print_result(100 * noun + verb)

'RESULT: 5741'

'TIME............................................................'
CPU times: user 295 ms, sys: 2.75 ms, total: 298 ms
Wall time: 298 ms


# 3

## setup

In [87]:
wires_input = get_level_input("03").splitlines()
wires_input = [x.split(',') for x in wires_input]

X = 1
Y = 0
CENTER_POINT = (0,0)

def md(p1, p2):
    return (abs(p1[X]-p2[X]) + abs(p1[Y]-p2[Y]))

## part one

In [88]:
%%time

def generate_wire(instructions):
    wire = [CENTER_POINT]
    for x in instructions:
        cur_point = wire[-1]
        wire += {
         'U': [(cur_point[Y]-i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'D': [(cur_point[Y]+i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'L': [(cur_point[Y], cur_point[X]-i) for i in range(1, int(x[1:])+1)],
         'R': [(cur_point[Y], cur_point[X]+i) for i in range(1, int(x[1:])+1)]
        }[x[0]]
    return wire
                                  
    
wires = [set(generate_wire(w_i))-{CENTER_POINT} for w_i in wires_input]
res = min([md(x, CENTER_POINT) for x in set.intersection(*wires)])
print_result(res)

'RESULT: 225'

'TIME............................................................'
CPU times: user 271 ms, sys: 5.11 ms, total: 276 ms
Wall time: 276 ms


## part two

In [142]:
%%time

wires = [generate_wire(w_i) for w_i in wires_input]
intersections = set.intersection(*[set(wire) for wire in wires])-{CENTER_POINT}
res = min([wires[0].index(x)+wires[1].index(x) for x in intersections])
print_result(res)

'RESULT: 35194'

'TIME............................................................'
CPU times: user 358 ms, sys: 4.02 ms, total: 362 ms
Wall time: 362 ms


In [ ]:
# 4

In [ ]:
## setup

In [140]:
r_low, r_high = map(int, get_level_input("04").split("-"))

def check_value(val, functions):
    num_list = [int(i) for i in list(str(val))]
    return (all(f(num_list) for f in functions))

## part one

In [143]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_double(l):
    return any(l[i] == l[i+1] for i in range(len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    if(check_value(num, [check_increase, check_double])): num_poss_pass+=1

print_result(num_poss_pass)

'RESULT: 1694'

'TIME............................................................'
CPU times: user 1.67 s, sys: 4.64 ms, total: 1.67 s
Wall time: 1.67 s


## part two 

In [144]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_ungrouped_double(l):
    l = [-1] + l +[-1]
    return any(l[i-1] != l[i] == l[i+1] != l[i+2] for i in range(1, len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    num_list = [int(i) for i in list(str(num))]
    if(check_value(num, [check_increase, check_ungrouped_double])): num_poss_pass += 1

print_result(num_poss_pass)

'RESULT: 1148'

'TIME............................................................'
CPU times: user 2.58 s, sys: 5.62 ms, total: 2.58 s
Wall time: 2.58 s


# 5

## setup

In [33]:
ADD = 1
MULT = 2
INPUT = 3
OUTPUT = 4
END = 99

operation_length = {ADD: 4, MULT: 4, INPUT: 2, OUTPUT: 2, END: 1}
operation_num_inputs = {ADD: 2, MULT: 2, INPUT: 0, OUTPUT: 0}
comp_string = get_level_input("05")
comp_string = list(map(int, comp_string.split(',')))

def print_computer(comp_string):
    for i in range(0, len(comp_string), 10):
        print(f"[{i}]    {comp_string[i:i+10]}")
    

## part one

In [38]:
def run_computer(comp_string):
    i=0
    last_output = 0
    print_computer(comp_string)
    while(True):
        opcode_and_settings = "0000"+str(comp_string[i])
        opcode = int(opcode_and_settings[-2:])
        if(opcode == END): return comp_string, last_output
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[i: i+window]
        i+=window
        
        output = win[-1]
        num_inputs = operation_num_inputs[opcode]
        inputs = [win[i+1] if input_modes[i] else comp_string[win[i+1]] for i in range(operation_num_inputs[opcode])]
        if(opcode == INPUT): 
            inputs = [int(input("provide input value: ")), 0]
            print("")
        if(opcode == OUTPUT): 
            print(f"OUTPUT: {output}")
            last_output = output
            continue
        print(f"opcode: {win[0]}")
        print(f"inputs: {inputs}")
        print(f"output: {output}")
        print(f"window: {win}\n")
        print("")
        comp_string[output] = {
            ADD: (inputs[0] + inputs[1]),
            MULT: (inputs[0] * inputs[1]),
            INPUT: inputs[0],
        }[opcode]
        print_computer(comp_string)
    
print_result(run_computer(list(comp_string))[1])

[0]    [3, 225, 1, 225, 6, 6, 1100, 1, 238, 225]
[10]    [104, 0, 1101, 86, 8, 225, 1101, 82, 69, 225]
[20]    [101, 36, 65, 224, 1001, 224, -106, 224, 4, 224]
[30]    [1002, 223, 8, 223, 1001, 224, 5, 224, 1, 223]
[40]    [224, 223, 102, 52, 148, 224, 101, -1144, 224, 224]
[50]    [4, 224, 1002, 223, 8, 223, 101, 1, 224, 224]
[60]    [1, 224, 223, 223, 1102, 70, 45, 225, 1002, 143]
[70]    [48, 224, 1001, 224, -1344, 224, 4, 224, 102, 8]
[80]    [223, 223, 101, 7, 224, 224, 1, 223, 224, 223]
[90]    [1101, 69, 75, 225, 1001, 18, 85, 224, 1001, 224]
[100]    [-154, 224, 4, 224, 102, 8, 223, 223, 101, 2]
[110]    [224, 224, 1, 224, 223, 223, 1101, 15, 59, 225]
[120]    [1102, 67, 42, 224, 101, -2814, 224, 224, 4, 224]
[130]    [1002, 223, 8, 223, 101, 3, 224, 224, 1, 223]
[140]    [224, 223, 1101, 28, 63, 225, 1101, 45, 22, 225]
[150]    [1101, 90, 16, 225, 2, 152, 92, 224, 1001, 224]
[160]    [-1200, 224, 4, 224, 102, 8, 223, 223, 101, 7]
[170]    [224, 224, 1, 223, 224, 223, 1101, 45,

provide input value:  1



opcode: 3
inputs: [1, 0]
output: 225
window: [3, 225]


[0]    [3, 225, 1, 225, 6, 6, 1100, 1, 238, 225]
[10]    [104, 0, 1101, 86, 8, 225, 1101, 82, 69, 225]
[20]    [101, 36, 65, 224, 1001, 224, -106, 224, 4, 224]
[30]    [1002, 223, 8, 223, 1001, 224, 5, 224, 1, 223]
[40]    [224, 223, 102, 52, 148, 224, 101, -1144, 224, 224]
[50]    [4, 224, 1002, 223, 8, 223, 101, 1, 224, 224]
[60]    [1, 224, 223, 223, 1102, 70, 45, 225, 1002, 143]
[70]    [48, 224, 1001, 224, -1344, 224, 4, 224, 102, 8]
[80]    [223, 223, 101, 7, 224, 224, 1, 223, 224, 223]
[90]    [1101, 69, 75, 225, 1001, 18, 85, 224, 1001, 224]
[100]    [-154, 224, 4, 224, 102, 8, 223, 223, 101, 2]
[110]    [224, 224, 1, 224, 223, 223, 1101, 15, 59, 225]
[120]    [1102, 67, 42, 224, 101, -2814, 224, 224, 4, 224]
[130]    [1002, 223, 8, 223, 101, 3, 224, 224, 1, 223]
[140]    [224, 223, 1101, 28, 63, 225, 1101, 45, 22, 225]
[150]    [1101, 90, 16, 225, 2, 152, 92, 224, 1001, 224]
[160]    [-1200, 224, 4, 224, 102, 8, 223, 223